In [ ]:
import os
import json
import requests
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

# Constants for API interaction
CSV_FILE = "chat_history.csv"
API_URL = "https://ladybug-keen-goldfish.ngrok-free.app/v1/chat/completions"

# Function to get chat completion from the API
def get_chat_completion(url, user_prompt):
    payload = {
        "user_prompt": user_prompt,
    }

    json_payload = json.dumps(payload)
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(url, data=json_payload, headers=headers)
        response.raise_for_status()
        response_data = response.json()
        # Adjust according to your response structure
        return response_data["choices"][0]["message"]["content"]
    except requests.exceptions.HTTPError as http_err:
        return f"HTTP error occurred: {http_err}"
    except requests.exceptions.RequestException as req_err:
        return f"Request Error occurred: {req_err}"
    except KeyError as key_err:
        return f"Key error occurred: {key_err}"
    except Exception as err:
        return f"An error occurred: {err}"

# Create interactive widgets for the chatbot
input_area = widgets.Textarea(
    value='',
    placeholder='Message ChatGPT',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

# Load chat history
try:
    chat_history_df = pd.read_csv(CSV_FILE)
except FileNotFoundError:
    chat_history_df = pd.DataFrame(columns=["ChatID", "Role", "Content"])

# Initialize chat session state
existing_sessions = chat_history_df["ChatID"].unique().tolist() if not chat_history_df.empty else ["Session 1"]
chat_names = existing_sessions if existing_sessions else ["Session 1"]
selected_chat = chat_names[0]

# Widgets for session management
chat_select = widgets.Dropdown(
    options=chat_names,
    description='Chat Sessions:',
    style={'description_width': 'initial'}  
)
new_session_button = widgets.Button(description="New Session")
send_button = widgets.Button(description="Send")
output_area = widgets.Output(layout={'border': '1px solid black'})

# Function to save chat history to CSV
def save_chat_history(df):
    df.to_csv(CSV_FILE, index=False)
    print("Chat history saved to CSV.")  

# Display chat messages
def display_chat_messages():
    output_area.clear_output()
    with output_area:
        messages = chat_history_df[chat_history_df.ChatID == selected_chat].to_dict("records")
        for message in messages:
            role = message["Role"].capitalize()
            content = message["Content"]
            display(HTML(f"<b>{role}:</b> {content}<br>"))

# Function to handle button click
def on_button_click(b):
    global chat_history_df, selected_chat

    question = input_area.value
    if not question.strip():
        return  # Don't do anything if the input is empty

    with output_area:
        # Display the user's question immediately
        display(HTML(f"<b>User:</b> {question}<br>"))

    # Fetch the assistant's response
    answer = get_chat_completion(API_URL, question)
    
    # Display the assistant's answer
    with output_area:
        display(HTML(f"<b>AI:</b> {answer}<br>" + "-"*50))

    # Create new DataFrames for user and AI messages
    new_data_user = pd.DataFrame([{
        "ChatID": selected_chat,
        "Role": "User",
        "Content": question
    }])

    new_data_ai = pd.DataFrame([{
        "ChatID": selected_chat,
        "Role": "AI",
        "Content": answer
    }])

    # Concatenate the new messages to the chat history
    chat_history_df = pd.concat([chat_history_df, new_data_user, new_data_ai], ignore_index=True)

    # Save updated chat history to CSV
    save_chat_history(chat_history_df)

    # Clear the input box after submission
    input_area.value = ""

# Function to handle new session creation
def create_new_session(b):
    global chat_names, selected_chat

    new_name = f"Session {len(chat_names) + 1}"
    chat_names.append(new_name)
    chat_select.options = chat_names
    selected_chat = new_name
    chat_select.value = new_name

    # Display the new chat session
    display_chat_messages()

# Update chat session dropdown options and selected chat
def update_chat_session(change):
    global selected_chat
    selected_chat = chat_select.value
    display_chat_messages()

# Attach functions to buttons
send_button.on_click(on_button_click)
new_session_button.on_click(create_new_session)
chat_select.observe(update_chat_session, names='value')

# Display widgets
display(chat_select, new_session_button, input_area, send_button, output_area)

# Initially display the messages
display_chat_messages()
